In [1]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")

#import src
import Cleanup
import Transformation
import FileProcessing

In [2]:
#define the objects to be called later. 
cleanup=Cleanup.Cleanup()
transform=Transformation.Transformation()
fileProcessing=FileProcessing.FileProcessing()

Read the File from Data folder and remove all duplicates. 

In [3]:
file_name='Next_Generation_Simulation__NGSIM__Vehicle_Trajectories_and_Supporting_Data'
ngsim=fileProcessing.read_input(file_name)

original File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\scripts
Data File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\data


Remove any Duplicate records from the original File. 

In [4]:
cleanup.remove_dups(ngsim)
ngsim[(ngsim['Vehicle_ID'] == 2876) & ( ngsim['Global_Time'] == 1118847795200) ]

704000 duplicate values have been removed


,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Width,...,D_Zone,Int_ID,Section_ID,Direction,Movement,Preceding,Following,Space_Headway,Time_Headway,Location
5616484,2876,8163,962,1118847795200,4.869,35.424,6451146.297,1873352.681,16.0,6.9,...,NaN,NaN,NaN,NaN,NaN,2871,0,77.85,2.01,us-101


Feet to Metres:

Below Four parametrs consider distance in Feet and they are converted to metre. Speed to Metre/ second from Feet/second and acceleration from feet/second Square to Metre/Second Square. 

Drop the columns we wont use in the model.
Add Relative time which is the time from the first timframe in the dataset. This is calculated by using the minimum value ofGlobal time(absolute time from 1970)

In [5]:
ngsim=transform.convert_feet_to_metre(ngsim)

In [6]:
ngsim[(ngsim['Vehicle_ID'] == 2876) & ( ngsim['Global_Time'] == 1118847795200) ]

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Width,...,D_Zone,Int_ID,Section_ID,Direction,Movement,Preceding,Following,Space_Headway,Time_Headway,Location
5616484,2876,8163,962,1118847795200,1.484071,10.797235,6451146.297,1873352.681,4.8768,6.9,...,NaN,NaN,NaN,NaN,NaN,2871,0,23.72868,2.01,us-101


In [7]:
ngsim.shape

(11146526, 25)

In [8]:
ngsim=transform.drop_not_required_columns(ngsim)

In [9]:
ngsim[(ngsim['Vehicle_ID'] == 2876) & ( ngsim['Global_Time'] == 1118847795200) ]

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,Preceding,Space_Headway,Time_Headway,Location
5616484,2876,8163,1118847795200,1.484071,10.797235,4.8768,2,11.804904,0.0,1,2871,23.72868,2.01,us-101


Create Placeholders for the user defined columns we will be populating later, to be used in the model. 
Map the Vehicle classes to their english equivalent. 
Filter the original file into US-101 and I-80 to perform and populate the required fields later. These files contain same values of Vehicle ID used for multiple vehicles based on the highway. 



In [10]:
#filtered_ngsim= ngsim[((ngsim['Location'] == 'us-101') | (ngsim['Location'] == 'i-80')) & (ngsim['Preceding']>0) & (ngsim['Following']>0) ]
ngsim=transform.create_column_placeholders(ngsim)

In [11]:
ngsim[(ngsim['Vehicle_ID'] == 2876) & ( ngsim['Global_Time'] == 1118847795200) ]

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,Space_Headway,Time_Headway,Location,Preceding_Vehicle_Class,Rear_to_Front_Space_Headway,Front_To_Rear_Time_Headway,Velocity Difference_Following-Preceding,Acceleration Difference_Following-Preceding,L-F_Pair,v_Class_Name
5616484,2876,8163,1118847795200,1.484071,10.797235,4.8768,2,11.804904,0.0,1,...,23.72868,2.01,us-101,NaN,NaN,NaN,NaN,NaN,2871-2876,Car


Split data into US-101 and I-80 specific dataframes.

In [12]:
filtered_ngsim_U,filtered_ngsim_I = transform.bifurcate_highways(ngsim)

Create Sets to use and create the L-F Pairs. 

In [13]:
v_Class_M_U,v_Class_C_U,v_Class_HV_U = transform.class_vehicle_sets(filtered_ngsim_U)
v_Class_M_I,v_Class_C_I,v_Class_HV_I = transform.class_vehicle_sets(filtered_ngsim_I)


In [14]:
filtered_ngsim_U[(filtered_ngsim_U['Vehicle_ID'] == 2876) & ( filtered_ngsim_U['Global_Time'] == 1118847795200) ]

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,Space_Headway,Time_Headway,Location,Preceding_Vehicle_Class,Rear_to_Front_Space_Headway,Front_To_Rear_Time_Headway,Velocity Difference_Following-Preceding,Acceleration Difference_Following-Preceding,L-F_Pair,v_Class_Name
5616484,2876,8163,1118847795200,1.484071,10.797235,4.8768,2,11.804904,0.0,1,...,23.72868,2.01,us-101,NaN,NaN,NaN,NaN,NaN,2871-2876,Car


Add the Preceding Vehicle Class to the new previous class column

In [15]:
filtered_ngsim_U=transform.preceding_vehicle_class(filtered_ngsim_U,v_Class_M_U,v_Class_C_U,v_Class_HV_U)
filtered_ngsim_I=transform.preceding_vehicle_class(filtered_ngsim_I,v_Class_M_I,v_Class_C_I,v_Class_HV_I)

Map the Vehicle Length of the previous vehicle into the row. 

In [16]:
filtered_ngsim_U[(filtered_ngsim_U['Vehicle_ID'] == 2876) & ( filtered_ngsim_U['Global_Time'] == 1118847795200) ].filter(like='pre')

""
5616484


In [17]:
filtered_ngsim_U=transform.preceding_vehicle_length(filtered_ngsim_U)
filtered_ngsim_I=transform.preceding_vehicle_length(filtered_ngsim_I)

In [18]:
filtered_ngsim_U[(filtered_ngsim_U['Vehicle_ID'] == 2876) & ( filtered_ngsim_U['Global_Time'] == 1118847795200) ]

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,Location,Preceding_Vehicle_Class,Rear_to_Front_Space_Headway,Front_To_Rear_Time_Headway,Velocity Difference_Following-Preceding,Acceleration Difference_Following-Preceding,L-F_Pair,v_Class_Name,Vehicle_combination,preceding_vehicle_length
5616484,2876,8163,1118847795200,1.484071,10.797235,4.8768,2,11.804904,0.0,1,...,us-101,Car,NaN,NaN,NaN,NaN,2871-2876,Car,Car-Car,3.9624


Convert the Front to Front Space Headway to Back Bumper (preceding Vehicle) to front Bumper(of Following ) Headway. This will give the actual distance between the vehicle ignoring the vehicle length.
Find the time it would take for the Following vehicle to reach the rear Bumper of the Preceding Vehicle. 

In [19]:
filtered_ngsim_U= transform.front_to_front_bumper_details_changed_to_rear_to_front_bumper_details(filtered_ngsim_U)
filtered_ngsim_I= transform.front_to_front_bumper_details_changed_to_rear_to_front_bumper_details(filtered_ngsim_I)

In [20]:
filtered_ngsim_U[(filtered_ngsim_U['Vehicle_ID'] == 2876) & ( filtered_ngsim_U['Global_Time'] == 1118847795200) ].filter(like='pre')

,preceding_vehicle_length
5616484,3.9624


In [21]:
filtered_ngsim_U.columns

Index(['Vehicle_ID', 'Frame_ID', 'Global_Time', 'Local_X', 'Local_Y',
       'v_length', 'v_Class', 'v_Vel', 'v_Acc', 'Lane_ID', 'Preceding',
       'Space_Headway', 'Time_Headway', 'Location', 'Preceding_Vehicle_Class',
       'Rear_to_Front_Space_Headway', 'Front_To_Rear_Time_Headway',
       'Velocity Difference_Following-Preceding',
       'Acceleration Difference_Following-Preceding', 'L-F_Pair',
       'v_Class_Name', 'Vehicle_combination', 'preceding_vehicle_length'],
      dtype='object')

Vehicle Combination of the Preceding and the Following Vehicle. 

In [22]:
filtered_ngsim_U=transform.map_preceding_vehicle_details(filtered_ngsim_U)
filtered_ngsim_I=transform.map_preceding_vehicle_details(filtered_ngsim_I)


0          us-101
3          us-101
4          us-101
6          us-101
7          us-101
            ...  
8704795    us-101
8704797    us-101
8704798    us-101
8704799    us-101
8704800    us-101
Name: Location, Length: 4098933, dtype: object::621 cars dont change lanes
0          us-101
3          us-101
4          us-101
6          us-101
7          us-101
            ...  
8704795    us-101
8704797    us-101
8704798    us-101
8704799    us-101
8704800    us-101
Name: Location, Length: 4098933, dtype: object::2226 cars Change lanes
2           i-80
10          i-80
12          i-80
13          i-80
14          i-80
            ... 
11850521    i-80
11850522    i-80
11850523    i-80
11850524    i-80
11850525    i-80
Name: Location, Length: 4566387, dtype: object::817 cars dont change lanes
2           i-80
10          i-80
12          i-80
13          i-80
14          i-80
            ... 
11850521    i-80
11850522    i-80
11850523    i-80
11850524    i-80
11850525    i-80
Name: Loc

In [23]:
filtered_ngsim_U[(filtered_ngsim_U['Vehicle_ID'] == 2876) & ( filtered_ngsim_U['Global_Time'] == 1118847795200) ].filter(like='pre')

,preceding_vehicle_length,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class
2610448,3.9624,10.77468,-1.301496,True,34.525306,2.0


In [24]:
filtered_ngsim_U=transform.map_pairs(filtered_ngsim_U)
filtered_ngsim_I=transform.map_pairs(filtered_ngsim_I)


In [25]:
print(filtered_ngsim_U[(filtered_ngsim_U['Vehicle_ID'] == 2876) & ( filtered_ngsim_U['Global_Time'] == 1118847795200) ].filter(like='pre'))

         preceding_vehicle_length  preceding_Vehicle_Velocity  \
1011507                    3.9624                    10.77468   

         preceding_Vehicle_Acceleration  preceding_car_lane_changes  \
1011507                       -1.301496                        True   

         preceding_Local_Y  preceding_v_Class  
1011507          34.525306                2.0  


In [26]:
filtered_ngsim_U=cleanup.filter_records_for_model(filtered_ngsim_U)
filtered_ngsim_I=cleanup.filter_records_for_model(filtered_ngsim_I)

dataset before Row Removal(4098933, 32)
978991 rows removed for the first and last 5 seconds of the cars that changed lanes
dataset before Row Removal(4566387, 32)
1258463 rows removed for the first and last 5 seconds of the cars that changed lanes


In [27]:
print(filtered_ngsim_U[(filtered_ngsim_U['Vehicle_ID'] == 2876) & ( filtered_ngsim_U['Global_Time'] == 1118847795200) ].filter(like='pre'))

         preceding_vehicle_length  preceding_Vehicle_Velocity  \
1011507                    3.9624                    10.77468   

         preceding_Vehicle_Acceleration  preceding_car_lane_changes  \
1011507                       -1.301496                        True   

         preceding_Local_Y  preceding_v_Class  
1011507          34.525306                2.0  



1. Velocity difference of the Lead minus Preceding Vehicle.
2. Acceleration difference of the Lead minus Preceding Vehicle.
3. Find the Pair details of the Lead_following 
4. Find the cumulative time the Pair runs within the highway as a pair. 

In [28]:
filtered_ngsim= fileProcessing.merge_files(filtered_ngsim_U , filtered_ngsim_I)

fileProcessing.export_file(filtered_ngsim,'Cleaned_NGSIM_Data')

3119942
3307924
6427866


True

In [29]:
filtered_ngsim.columns

Index(['Vehicle_ID', 'Frame_ID', 'Global_Time', 'Local_X', 'Local_Y',
       'v_length', 'v_Class', 'v_Vel', 'v_Acc', 'Lane_ID', 'Preceding',
       'Space_Headway', 'Time_Headway', 'Location', 'Preceding_Vehicle_Class',
       'Rear_to_Front_Space_Headway', 'Front_To_Rear_Time_Headway',
       'Velocity Difference_Following-Preceding',
       'Acceleration Difference_Following-Preceding', 'L-F_Pair',
       'v_Class_Name', 'Vehicle_combination', 'preceding_vehicle_length',
       'lane_changes', 'Prec_Vehicle_ID', 'preceding_Vehicle_Velocity',
       'preceding_Vehicle_Acceleration', 'preceding_car_lane_changes',
       'preceding_Local_Y', 'preceding_v_Class', 'pair_Time_Duration',
       'total_pair_duration'],
      dtype='object')

In [30]:
print(f"{filtered_ngsim[(filtered_ngsim['preceding_car_lane_changes'] == True) ]['Vehicle_ID'].unique().size} cars Change lanes")
print(f"{filtered_ngsim[(filtered_ngsim['preceding_car_lane_changes'] == False) ]['Vehicle_ID'].unique().size} cars DonT")

2750 cars Change lanes
1299 cars DonT


In [31]:
#filtered_ngsim[((filtered_ngsim['Vehicle_ID'] <= 100 )  ) & ((filtered_ngsim['Preceding'] <= 100 )  )].to_csv('classVERIFYFILE.CSV', index = False)

In [32]:
filtered_ngsim

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,pair_Time_Duration,total_pair_duration
0,5,8,1118846979700,12.127382,11.934139,5.18160,2,12.192000,0.000000,4,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.0,160.8
1,5,9,1118846979800,12.120982,13.153034,5.18160,2,12.192000,0.000000,4,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.1,160.8
2,5,10,1118846979900,12.114886,14.372539,5.18160,2,12.192000,0.000000,4,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.2,160.8
3,5,11,1118846980000,12.108485,15.591739,5.18160,2,12.192000,0.000000,4,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.3,160.8
4,5,12,1118846980100,12.102084,16.810634,5.18160,2,12.192000,0.000000,4,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.4,160.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4566341,2485,11650,1113438729900,16.713403,489.215278,4.20624,2,1.069848,0.332232,5,...,4.52628,True,2460,1.947672,0.826008,True,498.871342,2.0,128.0,128.4
4566342,2485,11651,1113438730000,16.714013,489.316776,4.20624,2,1.170432,1.487424,5,...,4.52628,True,2460,1.953768,0.015240,True,499.073424,2.0,128.1,128.4
4566344,2485,11652,1113438730100,16.714927,489.431990,4.20624,2,1.383792,2.868168,5,...,4.52628,True,2460,1.953768,0.000000,True,499.309949,2.0,128.2,128.4
4566346,2485,11653,1113438730200,16.717366,489.581342,4.20624,2,1.694688,3.413760,5,...,4.52628,True,2460,1.953768,0.000000,True,499.462349,2.0,128.3,128.4


In [33]:
filtered_ngsim[(filtered_ngsim['Vehicle_ID'] == 2876) & ( filtered_ngsim['Global_Time'] == 1118847795200)].filter(like='pre')

,preceding_vehicle_length,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class
1011507,3.9624,10.77468,-1.301496,True,34.525306,2.0


In [34]:
filtered_ngsim[(filtered_ngsim['Vehicle_ID'] == 2876)]

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,pair_Time_Duration,total_pair_duration
1011507,2876,8163,1118847795200,1.484071,10.797235,4.8768,2,11.804904,0.000000,1,...,3.9624,False,2871,10.774680,-1.301496,True,34.525306,2.0,0.0,68.7
1011614,2876,8164,1118847795300,1.477670,12.016740,4.8768,2,11.804904,0.000000,1,...,3.9624,False,2871,10.686288,-0.454152,True,35.590582,2.0,0.1,68.7
1011842,2876,8165,1118847795400,1.471574,13.235635,4.8768,2,11.804904,0.000000,1,...,3.9624,False,2871,10.658856,0.036576,True,36.653114,2.0,0.2,68.7
1012072,2876,8166,1118847795500,1.465174,14.454835,4.8768,2,11.804904,0.000000,1,...,3.9624,False,2871,10.661904,0.076200,True,37.718695,2.0,0.3,68.7
1012250,2876,8167,1118847795600,1.459992,15.521635,4.8768,2,11.804904,0.000000,1,...,3.9624,False,2871,10.668000,0.000000,True,38.785800,2.0,0.4,68.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1143301,2876,8930,1118847871900,2.339645,386.700065,4.8768,2,3.514344,0.000000,1,...,3.9624,False,2871,9.561576,3.227832,True,404.292816,2.0,68.3,68.7
1143446,2876,8931,1118847872000,2.325929,387.051499,4.8768,2,3.514344,0.000000,1,...,3.9624,False,2871,9.909048,3.413760,True,405.265738,2.0,68.4,68.7
1143667,2876,8932,1118847872100,2.312213,387.403238,4.8768,2,3.514344,0.000000,1,...,3.9624,False,2871,10.256520,3.246120,True,406.274626,2.0,68.5,68.7
1143839,2876,8933,1118847872200,2.298802,387.755282,4.8768,2,3.508248,-0.106680,1,...,3.9624,False,2871,10.500360,1.719072,True,407.326186,2.0,68.6,68.7


In [35]:
filtered_ngsim['L-F_Pair'].unique().shape

(9252,)

In [36]:
filtered_ngsim[((filtered_ngsim['Space_Headway'] == 0 ) & (filtered_ngsim['Preceding'] > 0 )) ]


,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,pair_Time_Duration,total_pair_duration
1113491,2964,8731,1118847852000,1.483766,94.405399,5.02920,2,2.618232,-3.413760,1,...,4.72440,False,2956,1.411224,1.298448,False,94.406923,2.0,29.7,89.6
2396215,2506,7263,1113437493200,16.258946,115.914526,4.35864,2,3.514344,-0.079248,5,...,4.66344,False,2512,2.819400,0.000000,False,115.914526,2.0,23.1,44.8
2553891,2491,7916,1113437558500,5.230673,182.539843,4.66344,2,3.453384,0.036576,2,...,4.51104,True,2486,3.200400,0.000000,False,182.540148,2.0,33.0,99.8
3196675,572,3292,1113437894100,5.464454,452.776438,4.35864,2,0.000000,0.000000,2,...,4.81584,True,566,0.749808,0.103632,True,452.777657,2.0,29.0,48.2
3664306,1748,6068,1113438171700,5.372100,22.727107,5.42544,2,7.504176,0.000000,2,...,4.35864,True,1734,7.620000,0.000000,True,22.727412,2.0,0.2,43.6


In [37]:

#filtered_ngsim[(filtered_ngsim['L-F_Pair'] == '15-13')  ]